In [1]:
!pip install -q simpletransformers
!pip install attacut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=49b698c96df876a8c18c79ce4c7259c1966c8111eab60df5988ffd6f5cf0e076
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [2]:
%pwd

'/kaggle/working'

In [3]:
dir = '/kaggle/input/nithan-chadok-name-entity-recognition'

In [4]:
import pandas as pd
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, pipeline
from attacut import tokenize
import torch
from sklearn.metrics import f1_score
import json
from datasets import load_dataset

2024-03-02 13:25:50.974726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 13:25:50.974819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 13:25:51.154277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


------

# Train model

In [5]:
# !tar -xvf /content/ner/AIFORTHAI-LST20Corpus.tar.gz -C /content

In [6]:
# #Load file in directory into variable lst20
# lst20 = load_dataset("lst20", data_dir="/content/LST20_Corpus")
# lst20

In [7]:
# #Convert to dataFrame
# train_df = pd.DataFrame(lst20['train'])
# validation_df = pd.DataFrame(lst20['validation'])
# test_df = pd.DataFrame(lst20['test'])
# train_df.head(3)

In [8]:
# #Column Filter only NER Tags
# df_filter = ['id', 'tokens', 'ner_tags']
# train_df = train_df[df_filter]
# validation_df = validation_df[df_filter]
# test_df = test_df[df_filter]
# train_df.head(3)

In [9]:
# # Define Ner Tags replace missing ne_list.txt
# NER_TAGS = [
#        "O",
#         "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "B_ORG",        "B_PER",        "B_TRM",        "B_TTL",
#        "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
#         "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "E_ORG",        "E_PER",        "E_TRM",        "E_TTL"]
# print(NER_TAGS)
# len(NER_TAGS)

In [10]:
# Simpletransformers Data Formats
# #Code NER Tags mapping
# #[sentenceid][token]
# print(train_df['ner_tags'][0][0])
# print(NER_TAGS[train_df['ner_tags'][0][0]])

In [11]:
# #Simple Transformer format
# #Append all word into the same dataFrame
# #Extract Data form Data Loader
# def convert_data_to_df(df):
#   data_df = pd.DataFrame()
#   sentence_id = []
#   words = []
#   labels = []

#   for sentence in range(len(df)):
#     for token in range(len(df['tokens'][sentence])):
#       sentence_id.append(sentence)
#       words.append(df['tokens'][sentence][token])
#       labels.append(NER_TAGS[df['ner_tags'][sentence][token]]) #Map 0 to "O", 1 to "B_BRN"

#   return pd.DataFrame(
#       {"sentence_id": sentence_id, "words": words, "labels": labels}
#   )

In [12]:
# train_data = convert_data_to_df(train_df)

# #Re-process to validate and test dataset
# eval_data = convert_data_to_df(validation_df )
# test_data = convert_data_to_df(test_df)
# train_data.head(9)

In [13]:
# Prepare Traning Medel

# import logging
# from simpletransformers.ner import NERModel, NERArgs
# import torch

# # Simple Transformer https://simpletransformers.ai/docs/ner-minimal-start/
# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

In [14]:
# data = {
#     'Model': ['ALBERT', 'BERT', 'BERTweet', 'BigBird', 'CamemBERT', 'DeBERTa', 'DeBERTa', 'DeBERTaV2', 'DistilBERT', 'ELECTRA', 'HerBERT', 'LayoutLM', 'LayoutLMv2', 'Longformer', 'MobileBERT', 'MPNet', 'RemBERT', 'RoBERTa', 'SqueezeBert', 'XLM', 'XLM-RoBERTa', 'XLNet'],
#     'Model code for NERModel': ['albert', 'bert', 'bertweet', 'bigbird', 'camembert', 'deberta', 'deberta', 'deberta-v2', 'distilbert', 'electra', 'herbert', 'layoutlm', 'layoutlmv2', 'longformer', 'mobilebert', 'mpnet', 'rembert', 'roberta', 'squeezebert', 'xlm', 'xlmroberta', 'xlnet']
# }
# modelType_df = pd.DataFrame(data)
# modelType_df

In [15]:
# max_seq_length = train_data['words'].str.len().max()
# print("Maximum length in column 'words':", max_seq_length)

In [16]:
# import torch
# torch.cuda.empty_cache()
# !rm -r /content/outputs

In [17]:
# # Configure the model
# ner_args = NERArgs()
# ner_args.train_batch_size = 512 #128, 192 is fit for GPU T4, 512 for A100
# ner_args.use_multiprocessing = True
# ner_args.evaluate_during_training = True
# ner_args.eval_batch_size = 1024
# ner_args.num_train_epochs = 30 #10
# ner_args.overwrite_output_dir = True

# model = NERModel(
#      "roberta", # Model Type
#      "xlm-roberta-base",  #Ner Pre-trained Model
#      args=ner_args, use_cuda=torch.cuda.is_available(), labels=NER_TAGS # Local Config
# )

In [18]:
# model.train_model(train_data, eval_data=eval_data)

In [19]:
# # Evaluate the model
# result, model_outputs, preds_list = model.eval_model(eval_data)

In [20]:
# #Copy best model folderinto gdrive folder
# !cp -r /content/outputs/best_model '/content/drive/MyDrive/Colab Notebooks/SuperAIHackathon/ner/best_model/xlm_roberta_best_models' #your destination

----------------------------

# Load Data Test

In [21]:
#Read test file
sample_submission = pd.read_csv(f'{dir}/sample_submission.csv')
df = pd.read_csv(f'{dir}/test.csv')
df['pred'] = sample_submission['pred']
sample_submission.head()


,i,pred
0,0,0.0
1,1,0.0
2,2,0.0
3,3,NaN
4,4,NaN


In [22]:
len(df)

65007

In [23]:
df.head()

,i,word,pred
0,0,ใน,0.0
1,1,สมัย,0.0
2,2,หนึ่ง,0.0
3,3,_,NaN
4,4,พระพุทธเจ้า,NaN


In [24]:
all_sentences = []
sent = ''
for i, row in df.iterrows():
    all_sentences.append(str(row['word']).replace("_","[!und:]"))
    df.at[i, 'start'] = len(sent)
    sent += all_sentences[-1]
    df.at[i, 'end'] = len(sent)
    sent += '_'
sentence = '_'.join(all_sentences)

In [25]:
df['end'] = 0
df['start'] = 0

In [26]:
sentence

'ใน_สมัย_หนึ่ง_[!und:]_พระพุทธเจ้า_ประทับ_อยู่_วัด_เชตวัน_[!und:]_เมือง_สาวัตถี_[!und:]_ทรง_ปรารภ_การ_ถวาย_บริขาร_ทุก_อย่าง_ของ_พ่อค้า_ชาวเมือง_คน_หนึ่ง_ได้_ตรัส_อดีต_นิทาน_มา_สาธก_[!und:]_ว่า_..._[!und:]_กาลครั้งหนึ่งนานมาแล้ว_[!und:]_พระโพธิสัตว์_เกิด_เป็น_กระต่าย_อาศัย_อยู่_ใน_ป่า_แห่ง_หนึ่ง_ท่ามกลาง_หุบเขา_และ_แม่น้ำ_ล้อมรอบ_[!und:]_มี_สัตว์_เป็น_เพื่อน_กัน_อีก_[!und:]_3_[!und:]_ตัว_[!und:]_คือ_[!und:]_ลิง_[!und:]_สุนัขจิ้งจอก_[!und:]_และ_นาก_[!und:]_สัตว์_ทั้ง_[!und:]_4_[!und:]_เป็น_สัตว์_มี_ศีลธรรม_[!und:]_ทุก_เย็น_จะ_มา_พบ_กัน_และ_ฟัง_โอวาท_ของ_กระต่าย_เสมอ_[!und:]_ต่อมา_วัน_หนึ่ง_[!und:]_กระต่าย_มอง_ดู_จันทร์_รู้_ว่า_พรุ่งนี้_จะ_เป็น_วัน_อุโบสถ_[!und:]_จึง_ให้_โอวาท_[!und:]_ว่า_[!und:]_""""_[!und:]_วันพรุ่งนี้_[!und:]_พวก_เรา_จง_พา_กัน_รักษาศีล_[!und:]_ให้ทาน_เถิด_[!und:]_เพราะ_มี_ผลบุญกุศล_มาก_[!und:]_ฉะนั้น_พวก_ท่าน_จง_เตรียม_อาหาร_ไว้_แบ่งปัน_คนขอทาน_เถิด_""""_[!und:]_สัตว์_ทั้ง_[!und:]_3_[!und:]_รับคำ_แล้ว_กลับ_ไป_ยัง_ที่อยู่_ของ_ตน_[!und:]_ครั้น_รุ่ง_ขึ้น_มี_นายพราน_คน_หนึ

In [27]:
sentence = sentence.replace('_',',')
sentence = sentence.replace("[!und:]","_")
sentence_comma = sentence.split(',')
sentence_comma

['ใน',
 'สมัย',
 'หนึ่ง',
 '_',
 'พระพุทธเจ้า',
 'ประทับ',
 'อยู่',
 'วัด',
 'เชตวัน',
 '_',
 'เมือง',
 'สาวัตถี',
 '_',
 'ทรง',
 'ปรารภ',
 'การ',
 'ถวาย',
 'บริขาร',
 'ทุก',
 'อย่าง',
 'ของ',
 'พ่อค้า',
 'ชาวเมือง',
 'คน',
 'หนึ่ง',
 'ได้',
 'ตรัส',
 'อดีต',
 'นิทาน',
 'มา',
 'สาธก',
 '_',
 'ว่า',
 '...',
 '_',
 'กาลครั้งหนึ่งนานมาแล้ว',
 '_',
 'พระโพธิสัตว์',
 'เกิด',
 'เป็น',
 'กระต่าย',
 'อาศัย',
 'อยู่',
 'ใน',
 'ป่า',
 'แห่ง',
 'หนึ่ง',
 'ท่ามกลาง',
 'หุบเขา',
 'และ',
 'แม่น้ำ',
 'ล้อมรอบ',
 '_',
 'มี',
 'สัตว์',
 'เป็น',
 'เพื่อน',
 'กัน',
 'อีก',
 '_',
 '3',
 '_',
 'ตัว',
 '_',
 'คือ',
 '_',
 'ลิง',
 '_',
 'สุนัขจิ้งจอก',
 '_',
 'และ',
 'นาก',
 '_',
 'สัตว์',
 'ทั้ง',
 '_',
 '4',
 '_',
 'เป็น',
 'สัตว์',
 'มี',
 'ศีลธรรม',
 '_',
 'ทุก',
 'เย็น',
 'จะ',
 'มา',
 'พบ',
 'กัน',
 'และ',
 'ฟัง',
 'โอวาท',
 'ของ',
 'กระต่าย',
 'เสมอ',
 '_',
 'ต่อมา',
 'วัน',
 'หนึ่ง',
 '_',
 'กระต่าย',
 'มอง',
 'ดู',
 'จันทร์',
 'รู้',
 'ว่า',
 'พรุ่งนี้',
 'จะ',
 'เป็น',
 'วัน',
 'อุโบสถ',
 '_',
 'จึ

In [28]:
# Sentence Tokenizer every 20 words
def split_into_sentences(tokens, tokens_per_sentence=20):
    sentences = []
    for i in range(0, len(tokens), tokens_per_sentence):
        sentence = tokens[i:i+tokens_per_sentence]
        sentences.append(sentence)
    return sentences


# def split_into_sentences(words):
#     sentences = []
#     current_sentence = []
#     word_count = 0

#     for word in words:
#         if word == '_':
#             sentences.append(current_sentence)
#             current_sentence = []
#             word_count = 0
#         else:
#             current_sentence.append(word)
#             word_count += 1
#             if word_count > 20:
#                 sentences.append(current_sentence)
#                 current_sentence = []
#                 word_count = 0

#     # Append the last sentence if it exists
#     if current_sentence:
#         sentences.append(current_sentence)

#     return sentences

# # Example long list of tokens
# tokens = ['นาง', 'สุดารัตน์', '_', 'เกยุราพันธ์', '_', 'รมว.', 'สาธารณสุข', '_', 'แถลง', 'ถึง', 'การ', ' ', 'นาง', 'อุไรวรรณ', '_', 'กระจ่างทิว', 'พันธุ์', 'นาง', 'อุไรวรรณ', 'กระจ่างทิว', 'ที่ประชุม']

In [29]:
my_token = split_into_sentences(sentence_comma)
my_token[0]

['ใน',
 'สมัย',
 'หนึ่ง',
 '_',
 'พระพุทธเจ้า',
 'ประทับ',
 'อยู่',
 'วัด',
 'เชตวัน',
 '_',
 'เมือง',
 'สาวัตถี',
 '_',
 'ทรง',
 'ปรารภ',
 'การ',
 'ถวาย',
 'บริขาร',
 'ทุก',
 'อย่าง']

In [30]:
#Check how many sentence
len(my_token)

3251

In [31]:
#Join all word in sentence seperate by space
sent_join = ' '.join(my_token[0])
type(sent_join)

str

In [32]:
sent_join

'ใน สมัย หนึ่ง _ พระพุทธเจ้า ประทับ อยู่ วัด เชตวัน _ เมือง สาวัตถี _ ทรง ปรารภ การ ถวาย บริขาร ทุก อย่าง'

In [33]:
#Check word inside list #Useful function
def data_inside(data_list):
    x = 0
    for i in range(len(data_list)):
        a = len(data_list[i])
        x = x+a
    return x

In [34]:
data_inside(my_token)

65008

# Prediction model

In [35]:
import pandas as pd
#Load and find unique NER Tags
tag_list = pd.read_csv(f'{dir}/tag_list.csv')[:-1]
tag_list

,tag,class
0,O,0
1,B_ORG,1
2,B_PER,2
3,B_LOC,3
4,B_MEA,4
5,I_DTM,5
6,I_ORG,6
7,E_ORG,7
8,I_PER,8
9,B_TTL,9


In [36]:
# Split the input string into lines, then split each line by comma to extract the 'tag' column only
# NER_TAGS = [row['tag'] for _, row in tag_list.iterrows()]
# NER_TAGS
# NER_TAGS = [
#        "O",
#         "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "B_ORG",        "B_PER",        "B_TRM",        "B_TTL",
#        "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
#         "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "E_ORG",        "E_PER",        "E_TRM",        "E_TTL"]
NER_TAGS = [
       "O",
        "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "O",        "B_PER",        "B_TRM",        "B_TTL",
       "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "O",        "I_PER",        "I_TRM",        "I_TTL",
        "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "O",        "E_PER",        "E_TRM",        "E_TTL"]
print(NER_TAGS)

['O', 'B_BRN', 'B_DES', 'B_DTM', 'B_LOC', 'B_MEA', 'B_NUM', 'O', 'B_PER', 'B_TRM', 'B_TTL', 'I_BRN', 'I_DES', 'I_DTM', 'I_LOC', 'I_MEA', 'I_NUM', 'O', 'I_PER', 'I_TRM', 'I_TTL', 'E_BRN', 'E_DES', 'E_DTM', 'E_LOC', 'E_MEA', 'E_NUM', 'O', 'E_PER', 'E_TRM', 'E_TTL']


# Predict model

In [37]:
from simpletransformers.ner import NERModel, NERArgs
import torch
print(torch.cuda.is_available())

# Test Model
ner_args = NERArgs()
ner_args.eval_batch_size = 128 #32 #40960
ner_args.use_multiprocessing = True
ner_args.max_seq_length = 81 # Fixed Requirement ##############################
model = NERModel(
     "auto", "/kaggle/input/xlm-roberta-base-best-model/xlm-roberta-base-best_model",
     args=ner_args, use_cuda=torch.cuda.is_available(),
     labels= NER_TAGS  # your latest model
)

True


In [38]:
# Make predictions with the model
predictions, raw_outputs = model.predict(my_token, False)

  0%|          | 0/5 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/26 [00:00<?, ?it/s]

In [39]:
# raw_outputs[:10]

In [40]:
#Check result sentence
len(predictions)

3251

In [41]:
#Check result data and type
predictions[0]

[{'ใน': 'O'},
 {'สมัย': 'O'},
 {'หนึ่ง': 'O'},
 {'_': 'O'},
 {'พระพุทธเจ้า': 'O'},
 {'ประทับ': 'O'},
 {'อยู่': 'O'},
 {'วัด': 'B_LOC'},
 {'เชตวัน': 'E_LOC'},
 {'_': 'O'},
 {'เมือง': 'B_LOC'},
 {'สาวัตถี': 'E_LOC'},
 {'_': 'O'},
 {'ทรง': 'O'},
 {'ปรารภ': 'O'},
 {'การ': 'O'},
 {'ถวาย': 'O'},
 {'บริขาร': 'O'},
 {'ทุก': 'O'},
 {'อย่าง': 'O'}]

In [42]:
# Function to check if a string represents a numeric value
def is_numeric(s):
    try:
        float(s)  # for int, long and float
    except ValueError:
        return False
    return True

# Modifying predictions based on the numeric status of words
for sentence in predictions:
    for word_dict in sentence:
        for word, pred in word_dict.items():
            if is_numeric(word):
                word_dict[word] = 'B_NUM'
            elif word == 'พระ' and word_dict[word]=='B_TTL':
                word_dict[word] = 'B_DES'


predictions

[[{'ใน': 'O'},
  {'สมัย': 'O'},
  {'หนึ่ง': 'O'},
  {'_': 'O'},
  {'พระพุทธเจ้า': 'O'},
  {'ประทับ': 'O'},
  {'อยู่': 'O'},
  {'วัด': 'B_LOC'},
  {'เชตวัน': 'E_LOC'},
  {'_': 'O'},
  {'เมือง': 'B_LOC'},
  {'สาวัตถี': 'E_LOC'},
  {'_': 'O'},
  {'ทรง': 'O'},
  {'ปรารภ': 'O'},
  {'การ': 'O'},
  {'ถวาย': 'O'},
  {'บริขาร': 'O'},
  {'ทุก': 'O'},
  {'อย่าง': 'O'}],
 [{'ของ': 'O'},
  {'พ่อค้า': 'O'},
  {'ชาวเมือง': 'O'},
  {'คน': 'O'},
  {'หนึ่ง': 'O'},
  {'ได้': 'O'},
  {'ตรัส': 'O'},
  {'อดีต': 'O'},
  {'นิทาน': 'O'},
  {'มา': 'O'},
  {'สาธก': 'O'},
  {'_': 'O'},
  {'ว่า': 'O'},
  {'...': 'O'},
  {'_': 'O'},
  {'กาลครั้งหนึ่งนานมาแล้ว': 'O'},
  {'_': 'O'},
  {'พระโพธิสัตว์': 'O'},
  {'เกิด': 'O'},
  {'เป็น': 'O'}],
 [{'กระต่าย': 'O'},
  {'อาศัย': 'O'},
  {'อยู่': 'O'},
  {'ใน': 'O'},
  {'ป่า': 'O'},
  {'แห่ง': 'O'},
  {'หนึ่ง': 'O'},
  {'ท่ามกลาง': 'O'},
  {'หุบเขา': 'O'},
  {'และ': 'O'},
  {'แม่น้ำ': 'O'},
  {'ล้อมรอบ': 'O'},
  {'_': 'O'},
  {'มี': 'O'},
  {'สัตว์': 'O'},
  {'เป็น': 'O'},


In [43]:
import pandas as pd

# Correct approach to flatten the list of dictionaries
flat_predictions = [(word, tag) for sentence in predictions for word_tag in sentence for word, tag in word_tag.items()]

# Convert to DataFrame
df_predictions = pd.DataFrame(flat_predictions, columns=['word', 'predict'])

# Display the first few rows to verify
df_predictions#.head()

,word,predict
0,ใน,O
1,สมัย,O
2,หนึ่ง,O
3,_,O
4,พระพุทธเจ้า,O
...,...,...
65003,ผู้,O
65004,สำรวม,O
65005,ดี,O
65006,แล้ว,O


In [44]:
from collections import Counter

# Assuming df_predictions is your DataFrame from the previous step

# Filter out rows where predict is not 'O'
non_o_predictions = df_predictions[df_predictions['predict'] != 'O']

# Extract the 'word' column from the filtered DataFrame
words_excluding_o = non_o_predictions['word']

# Use Counter to count the frequencies of each word
word_frequencies_excluding_o = Counter(words_excluding_o)

# Display the word frequencies, excluding those with prediction 'O'
word_frequencies_excluding_o

Counter({'เมือง': 180,
         'นาง': 107,
         'วัด': 94,
         'สาวัตถี': 89,
         'ละ': 86,
         'นาย': 84,
         'เชตวัน': 82,
         'สอง': 81,
         '2': 72,
         '4': 61,
         'ตัว': 59,
         'พาราณสี': 58,
         '7': 47,
         'วัน': 46,
         '_': 44,
         '3': 41,
         'พราหมณ์': 40,
         'นายพราน': 38,
         'พระเจ้า': 34,
         'พระ': 31,
         'คน': 31,
         'นก': 30,
         'คามณิจันท์': 29,
         'ป่า': 28,
         'หิมพานต์': 28,
         'สวนหลวง': 28,
         '1': 25,
         'ท่าน': 25,
         '500': 23,
         '6': 21,
         'ราชสีห์': 20,
         'เดียว': 20,
         '5': 18,
         'ท้าว': 17,
         'แม่น้ำ': 17,
         'พราหมณี': 16,
         'ช้าง': 15,
         'สักกะ': 14,
         'ราชคฤห์': 14,
         'ปูทอง': 13,
         'นกแขกเต้า': 13,
         'กา': 12,
         'คงคา': 12,
         'แพะ': 12,
         'พระสุบิน': 12,
         'ดาบส': 11,
         'อย่าง': 11

In [45]:
# The prediction information
# Step 1: Filter out rows where 'predict' is 'O'
filtered_df = df_predictions[df_predictions['predict'] != 'O']

# Step 2 & 3: Group by 'word' and 'predict', then count occurrences
word_prediction_frequencies = filtered_df.groupby(['word', 'predict']).size().reset_index(name='frequency')

# Display the results
word_prediction_frequencies


,word,predict,frequency
0,1,B_NUM,25
1,10,B_NUM,7
2,100,B_NUM,4
3,1000,B_NUM,4
4,100000,B_NUM,1
...,...,...,...
539,ไม้รัง,E_LOC,1
540,ไร่,B_MEA,2
541,๒-๓,B_MEA,5
542,๒-๓,B_NUM,1


In [46]:
#check word prediction
data_inside(predictions)

65008

In [47]:
#Extract data value from dict list
final_test_df = []
for i in range(len(predictions)):
# for i in range(3):
  for j in range(len(predictions[i])):
    data = predictions[i][j]
    value = data.values()
    final_test_df += value

In [48]:
#print result
print(len(final_test_df))
print(final_test_df)

65008
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_LOC', 'E_LOC', 'O', 'B_LOC', 'E_LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_NUM', 'O', 'B_MEA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_NUM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_NUM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_NUM', 'O', 'B_MEA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [49]:
final_result = pd.DataFrame(final_test_df)
final_result

,0
0,O
1,O
2,O
3,O
4,O
...,...
65003,O
65004,O
65005,O
65006,O


In [50]:
# Save result file to CSV
# final_result.to_csv('/kaggle/working/myresult.csv',index=False)

# Prepare submission file

In [51]:
submisstion_df = pd.read_csv(f'{dir}/sample_submission.csv')
submisstion_df

,i,pred
0,0,0.0
1,1,0.0
2,2,0.0
3,3,NaN
4,4,NaN
...,...,...
65002,65002,NaN
65003,65003,NaN
65004,65004,NaN
65005,65005,NaN


In [52]:
submisstion_df['pred'] = final_result
submisstion_df#.head(20)

,i,pred
0,0,O
1,1,O
2,2,O
3,3,O
4,4,O
...,...,...
65002,65002,O
65003,65003,O
65004,65004,O
65005,65005,O


In [53]:
NER_DICT = {row['tag']:row['class'] for _, row in tag_list.iterrows()}
NER_DICT

{'O': 0,
 'B_ORG': 1,
 'B_PER': 2,
 'B_LOC': 3,
 'B_MEA': 4,
 'I_DTM': 5,
 'I_ORG': 6,
 'E_ORG': 7,
 'I_PER': 8,
 'B_TTL': 9,
 'E_PER': 10,
 'B_DES': 11,
 'E_LOC': 12,
 'B_DTM': 13,
 'B_NUM': 14,
 'I_MEA': 15,
 'E_DTM': 16,
 'E_MEA': 17,
 'I_LOC': 18,
 'I_DES': 19,
 'E_DES': 20,
 'I_NUM': 21,
 'E_NUM': 22,
 'B_TRM': 23,
 'B_BRN': 24,
 'I_TRM': 25,
 'E_TRM': 26,
 'I_TTL': 27,
 'I_BRN': 28,
 'E_BRN': 29,
 'E_TTL': 30}

In [54]:
submisstion_df['pred'] = submisstion_df['pred'].map(NER_DICT)
submisstion_df

,i,pred
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
65002,65002,0
65003,65003,0
65004,65004,0
65005,65005,0


In [55]:
len(submisstion_df[submisstion_df['pred']==7])

0

In [56]:
submisstion_df.to_csv("/kaggle/working/submisstion_df.csv", index=False)